Segmentation and Clustering Assignment -2
Import required libraries

In [17]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')


Libraries imported.


Download Cananda Postal Details file in HTML format from WEB

In [4]:
!wget -q -O 'canada_postal_code.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Canada Post Code Data downloaded!')

Canada Post Code Data downloaded!


Import BeutifulSoup package bs4 for parsing HTML file

In [18]:
from bs4 import BeautifulSoup
print ("BeutifulSoup Imported")

BeutifulSoup Imported


In [ ]:
Open Canada postal code file and pass file handle to Beautiful Soup API

In [19]:
file_1 = open('canada_postal_code.html', "r")
canada_soup = BeautifulSoup(file_1, 'html.parser')

Get first table in HTML file, knowing first HTML table has all relevant data

In [20]:
 table = canada_soup.find_all('table')[0] # Grab the first table
 

Parse Table object and convert table row data into a DataFrame

In [21]:
suburb_table = pd.DataFrame()  
row_marker = 0
for row in table.find_all('tr'):
  column_marker = 0
  columns = row.find_all('td')
  for column in columns:
#            suburb_table.iloc[row_marker,column_marker] = column.get_text()
            suburb_table.set_value(row_marker, column_marker,column.get_text())
            column_marker +=1
  row_marker +=1  
suburb_table.columns = ['Postal Code', 'Borough', 'Neigbourhood']
suburb_table.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


,Postal Code,Borough,Neigbourhood
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
5,M5A,Downtown Toronto,Harbourfront\n


Strip Last Constrol Character from Neigbourhoos column

In [22]:
suburb_table['Neigbourhood'] =   \
suburb_table['Neigbourhood'].map(lambda x: str(x)[0:-1])
suburb_table.head()

,Postal Code,Borough,Neigbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


Remove rows for which Borough data is not available

In [23]:
suburb_table1= suburb_table.loc[suburb_table['Borough'] != 'Not assigned'] 
suburb_table1.head()

,Postal Code,Borough,Neigbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


join all neigbourhood in same postal code and Borough, Seperate by a comma

In [24]:
suburb_table2=suburb_table1.groupby(['Postal Code', 'Borough'],as_index=False, sort=False).agg(', '.join)
suburb_table2.head()


,Postal Code,Borough,Neigbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


Finally replace non Assigned neigbourhood by Borough

In [25]:
import numpy as np
#cond  = "suburb_table2.Neigbourhood == 'Not assigned' "
suburb_table2['Neigbourhood'] = np.where(suburb_table2.Neigbourhood == 'Not assigned' \
,suburb_table2.Borough, suburb_table2.Neigbourhood )
suburb_table2.head()
#suburb_table2.shape

,Postal Code,Borough,Neigbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Print Shape

In [26]:
suburb_table2.shape

(103, 3)

In [2]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    certifi-2018.11.29         |        py36_1000         145 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    ca-certificates-2018.11.29 |       ha4d7672_0         143 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

ImportError: cannot import name 'geocoders'

In [15]:
!wget -q -O 'postal_code_lat_long.csv'  http://cocl.us/Geospatial_data
print ("lat long downloaded")    

lat log downloaded


In [29]:
lat_long_data = pd.read_csv("postal_code_lat_long.csv") 
lat_long_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
merged_post_codes = pd.merge(suburb_table2, lat_long_data, on='Postal Code')
merged_post_codes.head()

,Postal Code,Borough,Neigbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
